# データサイエンス100本ノック（構造化データ加工編） - Polars

## はじめに

[100knocks-preprocess](https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess) からサンプルデータをダウンロードしてください。

- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
import polars as pl
pl.Config.set_fmt_str_lengths(100)

dtypes = {
    'customer_id': str,
    'gender_cd': str,
    'postal_cd': str,
    'application_store_cd': str,
    'status_cd': str,
    'category_major_cd': str,
    'category_medium_cd': str,
    'category_small_cd': str,
    'product_cd': str,
    'store_cd': str,
    'prefecture_cd': str,
    'tel_no': str,
    'postal_cd': str,
    'street': str
}
df_customer = pl.read_csv("data/customer.csv", dtypes=dtypes)
df_category = pl.read_csv("data/category.csv", dtypes=dtypes)
df_product = pl.read_csv("data/product.csv", dtypes=dtypes)
df_receipt = pl.read_csv("data/receipt.csv", dtypes=dtypes)
df_store = pl.read_csv("data/store.csv", dtypes=dtypes)
df_geocode = pl.read_csv("data/geocode.csv", dtypes=dtypes)

## 演習問題 001 ~ 010

### P-001

レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [18]:
df_receipt.head(5)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001""","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097""","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014""","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000""","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050""","""P060102007""",1,90


### P-002

レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [19]:
df_receipt.select('sales_ymd', 'customer_id', 'product_cd', 'amount').head(5)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20181103,"""CS006214000001""","""P070305012""",158
20181118,"""CS008415000097""","""P070701017""",81
20170712,"""CS028414000014""","""P060101005""",170
20190205,"""ZZ000000000000""","""P050301001""",25
20180821,"""CS025415000050""","""P060102007""",90


### P-003

レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更して抽出すること。

In [20]:
df_receipt.select(pl.col('sales_ymd').alias('sales_date'), 'customer_id', 'product_cd', 'amount').head(5)

sales_date,customer_id,product_cd,amount
i64,str,str,i64
20181103,"""CS006214000001""","""P070305012""",158
20181118,"""CS008415000097""","""P070701017""",81
20170712,"""CS028414000014""","""P060101005""",170
20190205,"""ZZ000000000000""","""P050301001""",25
20180821,"""CS025415000050""","""P060102007""",90


In [21]:
df_receipt.select('sales_ymd', 'customer_id', 'product_cd', 'amount').rename({'sales_ymd':'sales_date'}).head(5)

sales_date,customer_id,product_cd,amount
i64,str,str,i64
20181103,"""CS006214000001""","""P070305012""",158
20181118,"""CS008415000097""","""P070701017""",81
20170712,"""CS028414000014""","""P060101005""",170
20190205,"""ZZ000000000000""","""P050301001""",25
20180821,"""CS025415000050""","""P060102007""",90


### P-004

レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。

- 顧客ID（customer_id）が"CS018205000001"

In [22]:
df_receipt.filter(pl.col.customer_id == 'CS018205000001').select('sales_ymd', 'customer_id', 'product_cd', 'amount').head(5)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401012""",2200
20180414,"""CS018205000001""","""P060104007""",600
20170614,"""CS018205000001""","""P050206001""",990
20170614,"""CS018205000001""","""P060702015""",108
20190216,"""CS018205000001""","""P071005024""",102


### P-005

レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。

- 顧客ID（customer_id）が"CS018205000001"
- 売上金額（amount）が1,000以上

In [29]:
df_receipt.filter(
    (pl.col.customer_id == 'CS018205000001') & (pl.col.amount >= 1000)
).select(
    'sales_ymd', 'customer_id', 'product_cd', 'amount'
)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401012""",2200
20190226,"""CS018205000001""","""P071401020""",2200
20180911,"""CS018205000001""","""P071401005""",1100


In [28]:
df_receipt.filter(
    pl.col.customer_id.eq('CS018205000001').and_(pl.col.amount.ge(1000))
).select(
    'sales_ymd', 'customer_id', 'product_cd', 'amount'
)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401012""",2200
20190226,"""CS018205000001""","""P071401020""",2200
20180911,"""CS018205000001""","""P071401005""",1100


### P-006

レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。

- 顧客ID（customer_id）が"CS018205000001"
- 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [30]:
df_receipt.filter(
    pl.col.customer_id.eq('CS018205000001').and_(pl.col.amount.ge(1000).or_(pl.col.quantity >= 5))
).select(
    'sales_ymd', 'customer_id', 'product_cd', 'amount', 'quantity'
)

sales_ymd,customer_id,product_cd,amount,quantity
i64,str,str,i64,i64
20180911,"""CS018205000001""","""P071401012""",2200,1
20180414,"""CS018205000001""","""P060104007""",600,6
20170614,"""CS018205000001""","""P050206001""",990,5
20190226,"""CS018205000001""","""P071401020""",2200,1
20180911,"""CS018205000001""","""P071401005""",1100,1


### P-007

レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。

- 顧客ID（customer_id）が"CS018205000001"
- 売上金額（amount）が1,000以上2,000以下

In [34]:
df_receipt.filter(
    (pl.col.customer_id == 'CS018205000001') & pl.col.amount.is_between(1000, 2000)
).select(
    'sales_ymd', 'customer_id', 'product_cd', 'amount'
)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401005""",1100


### P-008

レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。

- 顧客ID（customer_id）が"CS018205000001"
- 商品コード（product_cd）が"P071401019"以外

In [35]:
df_receipt.filter(
    (pl.col.customer_id == 'CS018205000001') & (pl.col.product_cd != 'P071401019')
).select(
    'sales_ymd', 'customer_id', 'product_cd', 'amount'
)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401012""",2200
20180414,"""CS018205000001""","""P060104007""",600
20170614,"""CS018205000001""","""P050206001""",990
20170614,"""CS018205000001""","""P060702015""",108
20190216,"""CS018205000001""","""P071005024""",102
20180414,"""CS018205000001""","""P071101002""",278
20190226,"""CS018205000001""","""P070902035""",168
20190924,"""CS018205000001""","""P060805001""",495
20190226,"""CS018205000001""","""P071401020""",2200


### P-009

以下の処理において、出力結果を変えずにORをANDに書き換えよ。

```
df_store.query('not(prefecture_cd == "13" | floor_area > 900)')
```

In [36]:
df_store.filter(
    (pl.col.prefecture_cd != '13') & (pl.col.floor_area <= 900)
)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,str,str,str,str,str,f64,f64,f64
"""S14046""","""北山田店""","""14""","""神奈川県""","""神奈川県横浜市都筑区北山田一丁目""","""カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ""","""045-123-4049""",139.5916,35.56189,831.0
"""S14011""","""日吉本町店""","""14""","""神奈川県""","""神奈川県横浜市港北区日吉本町四丁目""","""カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ""","""045-123-4033""",139.6316,35.54655,890.0
"""S12013""","""習志野店""","""12""","""千葉県""","""千葉県習志野市芝園一丁目""","""チバケンナラシノシシバゾノイッチョウメ""","""047-123-4002""",140.022,35.66122,808.0


### P-010

店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [37]:
df_store.filter(pl.col.store_cd.str.starts_with('S14')).head(5)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,str,str,str,str,str,f64,f64,f64
"""S14010""","""菊名店""","""14""","""神奈川県""","""神奈川県横浜市港北区菊名一丁目""","""カナガワケンヨコハマシコウホククキクナイッチョウメ""","""045-123-4032""",139.6326,35.50049,1732.0
"""S14033""","""阿久和店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4043""",139.4961,35.45918,1495.0
"""S14036""","""相模原中央店""","""14""","""神奈川県""","""神奈川県相模原市中央二丁目""","""カナガワケンサガミハラシチュウオウニチョウメ""","""042-123-4045""",139.3716,35.57327,1679.0
"""S14040""","""長津田店""","""14""","""神奈川県""","""神奈川県横浜市緑区長津田みなみ台五丁目""","""カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ""","""045-123-4046""",139.4994,35.52398,1548.0
"""S14050""","""阿久和西店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4053""",139.4961,35.45918,1830.0


## 演習問題 011 ~ 020

### P-011

顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [38]:
df_customer.filter(pl.col.customer_id.str.ends_with('1')).head(5)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,str,str,str,i64,str,str,str,i64,str
"""CS037613000071""","""六角 雅彦""","""9""","""不明""","""1952-04-01""",66,"""136-0076""","""東京都江東区南砂**********""","""S13037""",20150414,"""0-00000000-0"""
"""CS028811000001""","""堀井 かおり""","""1""","""女性""","""1933-03-27""",86,"""245-0016""","""神奈川県横浜市泉区和泉町**********""","""S14028""",20160115,"""0-00000000-0"""
"""CS040412000191""","""川井 郁恵""","""1""","""女性""","""1977-01-05""",42,"""226-0021""","""神奈川県横浜市緑区北八朔町**********""","""S14040""",20151101,"""1-20091025-4"""
"""CS028314000011""","""小菅 あおい""","""1""","""女性""","""1983-11-26""",35,"""246-0038""","""神奈川県横浜市瀬谷区宮沢**********""","""S14028""",20151123,"""1-20080426-5"""
"""CS039212000051""","""藤島 恵梨香""","""1""","""女性""","""1997-02-03""",22,"""166-0001""","""東京都杉並区阿佐谷北**********""","""S13039""",20171121,"""1-20100215-4"""


### P-012

店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [40]:
df_store.filter(pl.col.address.str.contains("横浜市"))

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,str,str,str,str,str,f64,f64,f64
"""S14010""","""菊名店""","""14""","""神奈川県""","""神奈川県横浜市港北区菊名一丁目""","""カナガワケンヨコハマシコウホククキクナイッチョウメ""","""045-123-4032""",139.6326,35.50049,1732.0
"""S14033""","""阿久和店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4043""",139.4961,35.45918,1495.0
"""S14040""","""長津田店""","""14""","""神奈川県""","""神奈川県横浜市緑区長津田みなみ台五丁目""","""カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ""","""045-123-4046""",139.4994,35.52398,1548.0
"""S14050""","""阿久和西店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4053""",139.4961,35.45918,1830.0
"""S14028""","""二ツ橋店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区二ツ橋町""","""カナガワケンヨコハマシセヤクフタツバシチョウ""","""045-123-4042""",139.4963,35.46304,1574.0
"""S14012""","""本牧和田店""","""14""","""神奈川県""","""神奈川県横浜市中区本牧和田""","""カナガワケンヨコハマシナカクホンモクワダ""","""045-123-4034""",139.6582,35.42156,1341.0
"""S14046""","""北山田店""","""14""","""神奈川県""","""神奈川県横浜市都筑区北山田一丁目""","""カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ""","""045-123-4049""",139.5916,35.56189,831.0
"""S14011""","""日吉本町店""","""14""","""神奈川県""","""神奈川県横浜市港北区日吉本町四丁目""","""カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ""","""045-123-4033""",139.6316,35.54655,890.0
"""S14048""","""中川中央店""","""14""","""神奈川県""","""神奈川県横浜市都筑区中川中央二丁目""","""カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ""","""045-123-4051""",139.5758,35.54912,1657.0


### P-013

顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [43]:
df_customer.filter(pl.col.status_cd.str.contains('^[A-F]')).head(5)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,str,str,str,i64,str,str,str,i64,str
"""CS031415000172""","""宇多田 貴美子""","""1""","""女性""","""1976-10-04""",42,"""151-0053""","""東京都渋谷区代々木**********""","""S13031""",20150529,"""D-20100325-C"""
"""CS015414000103""","""奥野 陽子""","""1""","""女性""","""1977-08-09""",41,"""136-0073""","""東京都江東区北砂**********""","""S13015""",20150722,"""B-20100609-B"""
"""CS011215000048""","""芦田 沙耶""","""1""","""女性""","""1992-02-01""",27,"""223-0062""","""神奈川県横浜市港北区日吉本町**********""","""S14011""",20150228,"""C-20100421-9"""
"""CS029415000023""","""梅田 里穂""","""1""","""女性""","""1976-01-17""",43,"""279-0043""","""千葉県浦安市富士見**********""","""S12029""",20150610,"""D-20100918-E"""
"""CS035415000029""","""寺沢 真希""","""9""","""不明""","""1977-09-27""",41,"""158-0096""","""東京都世田谷区玉川台**********""","""S13035""",20141220,"""F-20101029-F"""


### P-014

顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [46]:
df_customer.filter(pl.col.status_cd.str.contains('[1-9]$')).head(5)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,str,str,str,i64,str,str,str,i64,str
"""CS001215000145""","""田崎 美紀""","""1""","""女性""","""1995-03-29""",24,"""144-0055""","""東京都大田区仲六郷**********""","""S13001""",20170605,"""6-20090929-2"""
"""CS033513000180""","""安斎 遥""","""1""","""女性""","""1962-07-11""",56,"""241-0823""","""神奈川県横浜市旭区善部町**********""","""S14033""",20150728,"""6-20080506-5"""
"""CS011215000048""","""芦田 沙耶""","""1""","""女性""","""1992-02-01""",27,"""223-0062""","""神奈川県横浜市港北区日吉本町**********""","""S14011""",20150228,"""C-20100421-9"""
"""CS040412000191""","""川井 郁恵""","""1""","""女性""","""1977-01-05""",42,"""226-0021""","""神奈川県横浜市緑区北八朔町**********""","""S14040""",20151101,"""1-20091025-4"""
"""CS009315000023""","""皆川 文世""","""1""","""女性""","""1980-04-15""",38,"""154-0012""","""東京都世田谷区駒沢**********""","""S13009""",20150319,"""5-20080322-1"""


### P-015

顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [47]:
df_customer.filter(pl.col.status_cd.str.contains('^[A-F].*[1-9]$')).head(5)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,str,str,str,i64,str,str,str,i64,str
"""CS011215000048""","""芦田 沙耶""","""1""","""女性""","""1992-02-01""",27,"""223-0062""","""神奈川県横浜市港北区日吉本町**********""","""S14011""",20150228,"""C-20100421-9"""
"""CS022513000105""","""島村 貴美子""","""1""","""女性""","""1962-03-12""",57,"""249-0002""","""神奈川県逗子市山の根**********""","""S14022""",20150320,"""A-20091115-7"""
"""CS001515000096""","""水野 陽子""","""9""","""不明""","""1960-11-29""",58,"""144-0053""","""東京都大田区蒲田本町**********""","""S13001""",20150614,"""A-20100724-7"""
"""CS013615000053""","""西脇 季衣""","""1""","""女性""","""1953-10-18""",65,"""261-0026""","""千葉県千葉市美浜区幕張西**********""","""S12013""",20150128,"""B-20100329-6"""
"""CS020412000161""","""小宮 薫""","""1""","""女性""","""1974-05-21""",44,"""174-0042""","""東京都板橋区東坂下**********""","""S13020""",20150822,"""B-20081021-3"""


### P-016

店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [48]:
df_store.filter(pl.col.tel_no.str.contains(r'^\d{3}-\d{3}-\d{4}$'))

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,str,str,str,str,str,f64,f64,f64
"""S12014""","""千草台店""","""12""","""千葉県""","""千葉県千葉市稲毛区千草台一丁目""","""チバケンチバシイナゲクチグサダイイッチョウメ""","""043-123-4003""",140.118,35.63559,1698.0
"""S13002""","""国分寺店""","""13""","""東京都""","""東京都国分寺市本多二丁目""","""トウキョウトコクブンジシホンダニチョウメ""","""042-123-4008""",139.4802,35.70566,1735.0
"""S14010""","""菊名店""","""14""","""神奈川県""","""神奈川県横浜市港北区菊名一丁目""","""カナガワケンヨコハマシコウホククキクナイッチョウメ""","""045-123-4032""",139.6326,35.50049,1732.0
"""S14033""","""阿久和店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4043""",139.4961,35.45918,1495.0
"""S14036""","""相模原中央店""","""14""","""神奈川県""","""神奈川県相模原市中央二丁目""","""カナガワケンサガミハラシチュウオウニチョウメ""","""042-123-4045""",139.3716,35.57327,1679.0
"""S14040""","""長津田店""","""14""","""神奈川県""","""神奈川県横浜市緑区長津田みなみ台五丁目""","""カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ""","""045-123-4046""",139.4994,35.52398,1548.0
"""S14050""","""阿久和西店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4053""",139.4961,35.45918,1830.0
"""S13052""","""森野店""","""13""","""東京都""","""東京都町田市森野三丁目""","""トウキョウトマチダシモリノサンチョウメ""","""042-123-4030""",139.4383,35.55293,1087.0
"""S14028""","""二ツ橋店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区二ツ橋町""","""カナガワケンヨコハマシセヤクフタツバシチョウ""","""045-123-4042""",139.4963,35.46304,1574.0


### P-017

顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [53]:
df_customer.sort(by='birth_day', descending=False).head()

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,str,str,str,i64,str,str,str,i64,str
"""CS003813000014""","""村山 菜々美""","""1""","""女性""","""1928-11-26""",90,"""182-0007""","""東京都調布市菊野台**********""","""S13003""",20160214,"""0-00000000-0"""
"""CS026813000004""","""吉村 朝陽""","""1""","""女性""","""1928-12-14""",90,"""251-0043""","""神奈川県藤沢市辻堂元町**********""","""S14026""",20150723,"""0-00000000-0"""
"""CS018811000003""","""熊沢 美里""","""1""","""女性""","""1929-01-07""",90,"""204-0004""","""東京都清瀬市野塩**********""","""S13018""",20150403,"""0-00000000-0"""
"""CS027803000004""","""内村 拓郎""","""0""","""男性""","""1929-01-12""",90,"""251-0031""","""神奈川県藤沢市鵠沼藤が谷**********""","""S14027""",20151227,"""0-00000000-0"""
"""CS013801000003""","""天野 拓郎""","""0""","""男性""","""1929-01-15""",90,"""274-0824""","""千葉県船橋市前原東**********""","""S12013""",20160120,"""0-00000000-0"""


In [52]:
df_customer.select(pl.col('customer_name', 'birth_day', 'age').sort_by('birth_day')).head()

customer_name,birth_day,age
str,str,i64
"""村山 菜々美""","""1928-11-26""",90
"""吉村 朝陽""","""1928-12-14""",90
"""熊沢 美里""","""1929-01-07""",90
"""内村 拓郎""","""1929-01-12""",90
"""天野 拓郎""","""1929-01-15""",90


### P-018

顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [54]:
df_customer.sort(by='birth_day', descending=True).head()

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,str,str,str,i64,str,str,str,i64,str
"""CS035114000004""","""大村 美里""","""1""","""女性""","""2007-11-25""",11,"""156-0053""","""東京都世田谷区桜**********""","""S13035""",20150619,"""6-20091205-6"""
"""CS022103000002""","""福山 はじめ""","""9""","""不明""","""2007-10-02""",11,"""249-0006""","""神奈川県逗子市逗子**********""","""S14022""",20160909,"""0-00000000-0"""
"""CS002113000009""","""柴田 真悠子""","""1""","""女性""","""2007-09-17""",11,"""184-0014""","""東京都小金井市貫井南町**********""","""S13002""",20160304,"""0-00000000-0"""
"""CS004115000014""","""松井 京子""","""1""","""女性""","""2007-08-09""",11,"""165-0031""","""東京都中野区上鷺宮**********""","""S13004""",20161120,"""1-20081231-1"""
"""CS002114000010""","""山内 遥""","""1""","""女性""","""2007-06-03""",11,"""184-0015""","""東京都小金井市貫井北町**********""","""S13002""",20160920,"""6-20100510-1"""


### P-019

レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [75]:
df_receipt.select(
    pl.col('customer_id', 'amount').sort_by('amount', descending=True)
).with_columns(pl.col.amount.rank(method='min', descending=True).alias('rank')
).head(10)

customer_id,amount,rank
str,i64,u32
"""CS011415000006""",10925,1
"""ZZ000000000000""",6800,2
"""CS028605000002""",5780,3
"""CS015515000034""",5480,4
"""ZZ000000000000""",5480,4
"""ZZ000000000000""",5480,4
"""ZZ000000000000""",5440,7
"""CS021515000089""",5440,7
"""CS015515000083""",5280,9


### P-020

レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [79]:
df_receipt.select(
    pl.col('customer_id', 'amount').sort_by('amount', descending=True)
).with_columns(pl.col.amount.rank(method='ordinal', descending=True).alias('rank')
).head(10)

customer_id,amount,rank
str,i64,u32
"""CS011415000006""",10925,1
"""ZZ000000000000""",6800,2
"""CS028605000002""",5780,3
"""CS015515000034""",5480,4
"""ZZ000000000000""",5480,5
"""ZZ000000000000""",5480,6
"""ZZ000000000000""",5440,7
"""CS021515000089""",5440,8
"""CS015515000083""",5280,9


## 演習問題 021 ~ 030

### P-021

レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [80]:
df_receipt.shape[0]

104681

In [82]:
len(df_receipt)

104681

### P-022

レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [84]:
df_receipt.select(pl.col.customer_id.n_unique()).item()

8307

### P-023

レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [91]:
df_receipt.group_by('store_cd').agg(pl.col('amount', 'quantity').sum()).sort(by='store_cd').head()

store_cd,amount,quantity
str,i64,i64
"""S12007""",638761,2099
"""S12013""",787513,2425
"""S12014""",725167,2358
"""S12029""",794741,2555
"""S12030""",684402,2403


### P-024

レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [100]:
df_receipt.group_by(
    pl.col.customer_id, maintain_order=True
).agg(
    pl.col('sales_ymd').max()
).head()

customer_id,sales_ymd
str,i64
"""CS006214000001""",20190908
"""CS008415000097""",20190417
"""CS028414000014""",20191023
"""ZZ000000000000""",20191031
"""CS025415000050""",20191008


In [102]:
df_receipt.group_by(
    pl.col.customer_id, maintain_order=True
).agg(
    pl.all().sort_by('sales_ymd', descending=True).first()
).head()

customer_id,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,product_cd,quantity,amount
str,i64,i64,str,i64,i64,str,i64,i64
"""CS006214000001""",20190908,1567900800,"""S14006""",112,1,"""P071302010""",1,770
"""CS008415000097""",20190417,1555459200,"""S13008""",1192,1,"""P090405001""",1,200
"""CS028414000014""",20191023,1571788800,"""S14028""",1122,2,"""P070301019""",1,225
"""ZZ000000000000""",20191031,1572480000,"""S13019""",112,1,"""P080102015""",1,30
"""CS025415000050""",20191008,1570492800,"""S14025""",1182,2,"""P080801009""",1,258


In [105]:
df_receipt.group_by(
    pl.col.customer_id, maintain_order=True
).agg(
    pl.all().get(pl.col.sales_ymd.arg_max())
).head()

customer_id,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,product_cd,quantity,amount
str,i64,i64,str,i64,i64,str,i64,i64
"""CS006214000001""",20190908,1567900800,"""S14006""",112,1,"""P071302010""",1,770
"""CS008415000097""",20190417,1555459200,"""S13008""",1192,1,"""P090405001""",1,200
"""CS028414000014""",20191023,1571788800,"""S14028""",1122,2,"""P070301019""",1,225
"""ZZ000000000000""",20191031,1572480000,"""S13019""",112,1,"""P080102015""",1,30
"""CS025415000050""",20191008,1570492800,"""S14025""",1182,2,"""P080801009""",1,258


### P-025

レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [3]:
df_receipt.group_by(
    pl.col.customer_id, maintain_order=True
).agg(
    pl.col('sales_ymd').min()
).head()

customer_id,sales_ymd
str,i64
"""CS006214000001""",20170509
"""CS008415000097""",20170328
"""CS028414000014""",20170403
"""ZZ000000000000""",20170101
"""CS025415000050""",20170619


### P-026

レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [5]:
df_receipt.lazy().group_by(
    'customer_id', maintain_order=True
).agg(
    pl.col.sales_ymd.min().alias('sales_ymd_min'),
    pl.col.sales_ymd.max().alias('sales_ymd_max'),    
).filter(
    pl.col.sales_ymd_min != pl.col.sales_ymd_max
).head(5).collect()

customer_id,sales_ymd_min,sales_ymd_max
str,i64,i64
"""CS006214000001""",20170509,20190908
"""CS008415000097""",20170328,20190417
"""CS028414000014""",20170403,20191023
"""ZZ000000000000""",20170101,20191031
"""CS025415000050""",20170619,20191008


### P-027

レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [6]:
df_receipt.group_by('store_cd').agg(
    pl.col.amount.mean()
).sort(by='amount', descending=True).head(5)

store_cd,amount
str,f64
"""S13052""",402.86747
"""S13015""",351.11196
"""S13003""",350.915519
"""S14010""",348.791262
"""S13001""",348.470386


### P-028

レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [7]:
df_receipt.group_by('store_cd').agg(
    pl.col.amount.median()
).sort(by='amount', descending=True).head(5)

store_cd,amount
str,f64
"""S13052""",190.0
"""S14010""",188.0
"""S14050""",185.0
"""S13003""",180.0
"""S13018""",180.0


### P-029

レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [34]:
mode = pl.col.product_cd.mode()
df_receipt.group_by('store_cd').agg(
    mode,
    (pl.col.product_cd == mode.first()).sum().alias('count')
).sort(by='store_cd').head(10)

store_cd,product_cd,count
str,list[str],u32
"""S12007""","[""P060303001""]",72
"""S12013""","[""P060303001""]",107
"""S12014""","[""P060303001""]",65
"""S12029""","[""P060303001""]",92
"""S12030""","[""P060303001""]",115
"""S13001""","[""P060303001""]",67
"""S13002""","[""P060303001""]",78
"""S13003""","[""P071401001""]",65
"""S13004""","[""P060303001""]",88


In [21]:
df_receipt.group_by('store_cd', 'product_cd').agg(
    pl.count()
).filter(
    (pl.col.count == pl.col.count.max()).over('store_cd')
).sort(by='store_cd').head(10)

store_cd,product_cd,count
str,str,u32
"""S12007""","""P060303001""",72
"""S12013""","""P060303001""",107
"""S12014""","""P060303001""",65
"""S12029""","""P060303001""",92
"""S12030""","""P060303001""",115
"""S13001""","""P060303001""",67
"""S13002""","""P060303001""",78
"""S13003""","""P071401001""",65
"""S13004""","""P060303001""",88


### P-030

レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [36]:
df_receipt.group_by('store_cd').agg(
    vars_amount = pl.col.amount.var(ddof=0)
).sort(by='vars_amount', descending=True).head()

store_cd,vars_amount
str,f64
"""S13052""",440088.701311
"""S14011""",306314.558164
"""S14034""",296920.081011
"""S13001""",295431.993329
"""S13015""",295294.361116


## 演習問題 031 ~ 040

### P-031

レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [38]:
df_receipt.group_by('store_cd').agg(
    std_amount = pl.col.amount.std(ddof=0)
).sort(by='std_amount', descending=True).head()

store_cd,std_amount
str,f64
"""S13052""",663.391816
"""S14011""",553.456916
"""S14034""",544.903736
"""S13001""",543.536561
"""S13015""",543.409938


### P-032

レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [41]:
df_receipt.select(
    **{f'amount_{per}per': pl.col.amount.quantile(per / 100) for per in [25, 50, 75, 100]}
)

amount_25per,amount_50per,amount_75per,amount_100per
f64,f64,f64,f64
102.0,170.0,288.0,10925.0


### P-033

レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [44]:
df_receipt.group_by('store_cd', maintain_order=True).agg(
    avg_amount = pl.col.amount.mean()
).filter(pl.col.avg_amount >= 330)

store_cd,avg_amount
str,f64
"""S13003""",350.915519
"""S13020""",337.879932
"""S14026""",332.340588
"""S13015""",351.11196
"""S14045""",330.082073
"""S13019""",330.208616
"""S14011""",335.718333
"""S13004""",330.943949
"""S14010""",348.791262


### P-034

レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [48]:
df_receipt.filter(
    pl.col.customer_id.str.starts_with('Z').not_()
).group_by('customer_id').agg(
    pl.col.amount.sum()
).select(pl.col.amount.mean()).item()

2547.742234529256

### P-035

レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [54]:
df_receipt.lazy().filter(
    pl.col.customer_id.str.starts_with('Z').not_()
).group_by('customer_id').agg(
    sum_amount = pl.col.amount.sum()
).filter(
    pl.col.sum_amount >= pl.col.sum_amount.mean()
).collect().head(10)

customer_id,sum_amount
str,i64
"""CS030214000025""",3333
"""CS008515000084""",8064
"""CS020113000002""",5775
"""CS020415000197""",8426
"""CS035415000205""",7630
"""CS020514000069""",2654
"""CS001605000009""",18925
"""CS011115000004""",3351
"""CS018414000116""",4837


### P-036

レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [55]:
df_receipt.join(df_store.select('store_cd', 'store_name'), on='store_cd').head(10)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
i64,i64,str,i64,i64,str,str,i64,i64,str
20181103,1541203200,"""S14006""",112,1,"""CS006214000001""","""P070305012""",1,158,"""葛が谷店"""
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097""","""P070701017""",1,81,"""成城店"""
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014""","""P060101005""",1,170,"""二ツ橋店"""
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000""","""P050301001""",1,25,"""新山下店"""
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050""","""P060102007""",1,90,"""大和店"""
20190605,1559692800,"""S13003""",1112,1,"""CS003515000195""","""P050102002""",1,138,"""狛江店"""
20181205,1543968000,"""S14024""",1102,2,"""CS024514000042""","""P080101005""",1,30,"""三田店"""
20190922,1569110400,"""S14040""",1102,1,"""CS040415000178""","""P070501004""",1,128,"""長津田店"""
20170504,1493856000,"""S13020""",1112,2,"""ZZ000000000000""","""P071302010""",1,770,"""十条仲原店"""


### P-037

商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [56]:
df_product.join(df_category.select('category_small_cd', 'category_small_name'), on='category_small_cd').head(10)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
str,str,str,str,i64,i64,str
"""P040101001""","""04""","""0401""","""040101""",198,149,"""弁当類"""
"""P040101002""","""04""","""0401""","""040101""",218,164,"""弁当類"""
"""P040101003""","""04""","""0401""","""040101""",230,173,"""弁当類"""
"""P040101004""","""04""","""0401""","""040101""",248,186,"""弁当類"""
"""P040101005""","""04""","""0401""","""040101""",268,201,"""弁当類"""
"""P040101006""","""04""","""0401""","""040101""",298,224,"""弁当類"""
"""P040101007""","""04""","""0401""","""040101""",338,254,"""弁当類"""
"""P040101008""","""04""","""0401""","""040101""",420,315,"""弁当類"""
"""P040101009""","""04""","""0401""","""040101""",498,374,"""弁当類"""


### P-038

顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [68]:
df_customer.lazy().filter(
    (pl.col.gender_cd == "1") & (pl.col.customer_id.str.starts_with('Z').not_())
).select(
    'customer_id'
).join(
    df_receipt.lazy().group_by('customer_id').agg(
        sum_amount = pl.col.amount.sum()
    ),
    on='customer_id',
    how='left'
).with_columns(
    pl.col.sum_amount.fill_null(0)
).head(10).collect()

customer_id,sum_amount
str,i64
"""CS021313000114""",0
"""CS031415000172""",5088
"""CS028811000001""",0
"""CS001215000145""",875
"""CS015414000103""",3122
"""CS033513000180""",868
"""CS035614000014""",0
"""CS011215000048""",3444
"""CS009413000079""",0


### P-039

レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [86]:
df_data = df_receipt.filter(
    pl.col.customer_id.str.starts_with('Z').not_()
)

group = df_data.group_by('customer_id')

df_cnt = group.agg(
    come_days = pl.col.sales_ymd.n_unique()
).top_k(20, by='come_days')

df_sum = group.agg(
    buy_amount = pl.col.amount.sum()
).top_k(20, by='buy_amount')

df_cnt.join(df_sum, on='customer_id', how='outer_coalesce')

customer_id,come_days,buy_amount
str,u32,i64
"""CS017415000097""",20,23086
"""CS015415000185""",22,20153
"""CS031414000051""",19,19202
"""CS028415000007""",21,19127
"""CS001605000009""",null,18925
"""CS010214000010""",22,18585
"""CS016415000141""",20,18372
"""CS006515000023""",null,18372
"""CS011414000106""",null,18338


### P-040

全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [92]:
df_store.join(df_product, how='cross').select(pl.count()).item()

531590

## 演習問題 041 ~ 050

### P-041

レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [100]:
df_receipt.lazy().group_by('sales_ymd').agg(
    pl.col.amount.sum()
).sort(
    by='sales_ymd'
).with_columns(
    pl.all().shift().name.prefix('lag_')    
).with_columns(
    diff_amount = pl.col.amount - pl.col.lag_amount
).head(10).collect()

sales_ymd,amount,lag_sales_ymd,lag_amount,diff_amount
i64,i64,i64,i64,i64
20170101,33723,null,null,null
20170102,24165,20170101,33723,-9558
20170103,27503,20170102,24165,3338
20170104,36165,20170103,27503,8662
20170105,37830,20170104,36165,1665
20170106,32387,20170105,37830,-5443
20170107,23415,20170106,32387,-8972
20170108,24737,20170107,23415,1322
20170109,26718,20170108,24737,1981


### P-042

レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [11]:
df_tmp = df_receipt.group_by(
    'sales_ymd'
).agg(
    pl.col.amount.sum()
).sort(
    by='sales_ymd'
).with_columns(
    index=pl.int_range(0, pl.len(), dtype=pl.Int32)
)

df_rolling = df_tmp.rolling('index', period='3i', offset='-4i').agg(
    pl.col.sales_ymd, 
    pl.col.amount
)

df_tmp.join(
    df_rolling, on='index', suffix='_lag'
).explode(
    pl.col('^*._lag$')
).drop(
    'index'
).head(10)

sales_ymd,amount,sales_ymd_lag,amount_lag
i64,i64,i64,i64
20170101,33723,null,null
20170102,24165,20170101,33723
20170103,27503,20170101,33723
20170103,27503,20170102,24165
20170104,36165,20170101,33723
20170104,36165,20170102,24165
20170104,36165,20170103,27503
20170105,37830,20170102,24165
20170105,37830,20170103,27503


### P-043

レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。

ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [2]:
df_sales_summary = df_receipt.join(
    df_customer, on='customer_id', how='inner'
).group_by(
    (pl.col.age // 10) * 10, 'gender_cd'
).agg(
    pl.col.amount.sum()
).sort(
    by=['age', 'gender_cd']
).with_columns(
    pl.col.gender_cd.replace({'0':'male', '1':'female', '9':'unknown'})
).pivot(
    index='age',
    columns='gender_cd',
    values='amount'
)
df_sales_summary

age,male,female,unknown
i64,i64,i64,i64
10,1591,149836,4317
20,72940,1363724,44328
30,177322,693047,50441
40,19355,9320791,483512
50,54320,6685192,342923
60,272469,987741,71418
70,13435,29764,2427
80,46360,262923,5111
90,null,6260,null


### P-044

043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [7]:
df_sales_summary.melt(
    id_vars='age', variable_name='gender_cd', value_name='amount'
).with_columns(
    pl.col.gender_cd.replace({'male':'00', 'female':'01', 'unknown':'99'})    
).head()

age,gender_cd,amount
i64,str,i64
10,"""00""",1591
20,"""00""",72940
30,"""00""",177322
40,"""00""",19355
50,"""00""",54320


### P-045

顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [12]:
df_customer.select(
    pl.col.customer_id,
    pl.col.birth_day.str.replace_all('-', '')
).head()

customer_id,birth_day
str,str
"""CS021313000114""","""19810429"""
"""CS037613000071""","""19520401"""
"""CS031415000172""","""19761004"""
"""CS028811000001""","""19330327"""
"""CS001215000145""","""19950329"""


### P-046

顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [17]:
df_customer.select(
    pl.col.customer_id,
    pl.col.application_date.cast(str).str.strptime(pl.Date, '%Y%m%d')
).head()

customer_id,application_date
str,date
"""CS021313000114""",2015-09-05
"""CS037613000071""",2015-04-14
"""CS031415000172""",2015-05-29
"""CS028811000001""",2016-01-15
"""CS001215000145""",2017-06-05


### P-047

レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [21]:
# cast(str)以外の方法
df_receipt.select(
    pl.col('receipt_no', 'receipt_sub_no'),
    pl.date(
        pl.col.sales_ymd // 10000, 
        pl.col.sales_ymd % 10000 // 100, 
        pl.col.sales_ymd % 100
    ).alias('sales_ymd')
).head()

receipt_no,receipt_sub_no,sales_ymd
i64,i64,date
112,1,2018-11-03
1132,2,2018-11-18
1102,1,2017-07-12
1132,1,2019-02-05
1102,2,2018-08-21


### P-048

レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [26]:
df_receipt.select(
    pl.col('receipt_no', 'receipt_sub_no'),
    pl.from_epoch(pl.col.sales_epoch).dt.date()
).head()

receipt_no,receipt_sub_no,sales_epoch
i64,i64,date
112,1,2018-11-03
1132,2,2018-11-18
1102,1,2017-07-12
1132,1,2019-02-05
1102,2,2018-08-21


### P-049

レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [27]:
df_receipt.select(
    pl.col('receipt_no', 'receipt_sub_no'),
    pl.from_epoch(pl.col.sales_epoch).dt.year()
).head()

receipt_no,receipt_sub_no,sales_epoch
i64,i64,i32
112,1,2018
1132,2,2018
1102,1,2017
1132,1,2019
1102,2,2018


### P-050

レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [32]:
df_receipt.select(
    pl.col('receipt_no', 'receipt_sub_no'),
    pl.from_epoch(pl.col.sales_epoch).dt.month().cast(str).str.pad_start(2, '0')
).head()

receipt_no,receipt_sub_no,sales_epoch
i64,i64,str
112,1,"""11"""
1132,2,"""11"""
1102,1,"""07"""
1132,1,"""02"""
1102,2,"""08"""


## 演習問題 051 ~ 060

### P-051

レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [33]:
df_receipt.select(
    pl.col('receipt_no', 'receipt_sub_no'),
    pl.from_epoch(pl.col.sales_epoch).dt.day().cast(str).str.pad_start(2, '0')
).head()

receipt_no,receipt_sub_no,sales_epoch
i64,i64,str
112,1,"""03"""
1132,2,"""18"""
1102,1,"""12"""
1132,1,"""05"""
1102,2,"""21"""


### P-052

レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [8]:
df_receipt.filter(
    pl.col.customer_id.str.starts_with('Z').not_()
).group_by(
    'customer_id', maintain_order=True
).agg(
    sum_amount=pl.col.amount.sum()
).with_columns(
    sales_flg=pl.when(pl.col.sum_amount >= 2000).then(1).otherwise(0)
).head()

customer_id,sum_amount,sales_flg
str,i64,i32
"""CS006214000001""",7364,1
"""CS008415000097""",1895,0
"""CS028414000014""",6222,1
"""CS025415000050""",5736,1
"""CS003515000195""",5412,1


### P-053

顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [19]:
df_receipt.join(
    df_customer.select(
        'customer_id',
        postal_flg=pl.col.postal_cd.str.slice(0, 3).cast(pl.UInt16).is_between(100, 209).cast(pl.UInt8)
    ),
    on='customer_id',
    how='inner'
).group_by(
    'postal_flg'
).agg(
    customer_cnt=pl.col.customer_id.n_unique()
)

postal_flg,customer_cnt
u8,u32
0,3906
1,4400


### P-054

顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [32]:
names = ['埼玉県', '千葉県', '東京都', '神奈川県']
codes = [11, 12, 13, 14]
df_customer.select(
    'customer_id', 'address',
    prefecture_cd=pl.col.address.str.extract('^(.*?[都道府県])').replace(names, codes)
).head()

customer_id,address,prefecture_cd
str,str,str
"""CS021313000114""","""神奈川県伊勢原市粟窪**********""","""14"""
"""CS037613000071""","""東京都江東区南砂**********""","""13"""
"""CS031415000172""","""東京都渋谷区代々木**********""","""13"""
"""CS028811000001""","""神奈川県横浜市泉区和泉町**********""","""14"""
"""CS001215000145""","""東京都大田区仲六郷**********""","""13"""


### P-055

レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。

- 最小値以上第1四分位未満 ・・・ 1を付与
- 第1四分位以上第2四分位未満 ・・・ 2を付与
- 第2四分位以上第3四分位未満 ・・・ 3を付与
- 第3四分位以上 ・・・ 4を付与

In [43]:
df_receipt.group_by(
    'customer_id', maintain_order=True
).agg(
    sum_amount=pl.col.amount.sum()
).with_columns(
    pct_group=pl.col.sum_amount.qcut([0.25, 0.5, 0.75], labels=['1', '2', '3', '4'])
).head(10)

customer_id,sum_amount,pct_group
str,i64,cat
"""CS006214000001""",7364,"""4"""
"""CS008415000097""",1895,"""3"""
"""CS028414000014""",6222,"""4"""
"""ZZ000000000000""",12395003,"""4"""
"""CS025415000050""",5736,"""4"""
"""CS003515000195""",5412,"""4"""
"""CS024514000042""",533,"""1"""
"""CS040415000178""",6149,"""4"""
"""CS027514000015""",2788,"""3"""


### P-056

顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [3]:
df_customer.select(
    'customer_id', 'birth_day',
    (pl.col.age // 10 * 10).clip(upper_bound=60)
).head(10)

customer_id,birth_day,age
str,str,i64
"""CS021313000114""","""1981-04-29""",30
"""CS037613000071""","""1952-04-01""",60
"""CS031415000172""","""1976-10-04""",40
"""CS028811000001""","""1933-03-27""",60
"""CS001215000145""","""1995-03-29""",20
"""CS020401000016""","""1974-09-15""",40
"""CS015414000103""","""1977-08-09""",40
"""CS029403000008""","""1973-08-17""",40
"""CS015804000004""","""1931-05-02""",60


### P-057

056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [9]:
df_customer.select(
    'customer_id', 'birth_day',
    gender_era=pl.col.gender_cd + ((pl.col.age // 10 * 10).clip(upper_bound=60) + 100).cast(str).str.slice(1, 2)
).head(10)

customer_id,birth_day,gender_era
str,str,str
"""CS021313000114""","""1981-04-29""","""130"""
"""CS037613000071""","""1952-04-01""","""960"""
"""CS031415000172""","""1976-10-04""","""140"""
"""CS028811000001""","""1933-03-27""","""160"""
"""CS001215000145""","""1995-03-29""","""120"""
"""CS020401000016""","""1974-09-15""","""040"""
"""CS015414000103""","""1977-08-09""","""140"""
"""CS029403000008""","""1973-08-17""","""040"""
"""CS015804000004""","""1931-05-02""","""060"""


### P-058

顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [14]:
pl.concat([
    df_customer.select('customer_id'), 
    df_customer.get_column('gender_cd').to_dummies()
], how='horizontal').head()

customer_id,gender_cd_0,gender_cd_1,gender_cd_9
str,u8,u8,u8
"""CS021313000114""",0,1,0
"""CS037613000071""",0,0,1
"""CS031415000172""",0,1,0
"""CS028811000001""",0,1,0
"""CS001215000145""",0,1,0


### P-059

レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

TIPS:
- query()の引数engineで'python'か'numexpr'かを選択でき、デフォルトはインストールされていればnumexprが、無ければpythonが使われます。さらに、文字列メソッドはengine='python'でないとquery()内で使えません。


In [19]:
df_receipt.filter(
    pl.col.customer_id.str.starts_with('Z').not_()    
).group_by(
    'customer_id'
).agg(
    sum_amount=pl.col.amount.sum()
).with_columns(
    std_amount=(pl.col.sum_amount - pl.col.sum_amount.mean()) / pl.col.sum_amount.std()
).head(10)

customer_id,sum_amount,std_amount
str,i64,f64
"""CS022511000034""",388,-0.793826
"""CS027614000007""",1304,-0.457145
"""CS017512000149""",338,-0.812204
"""CS032314000036""",1978,-0.209412
"""CS020414000094""",6466,1.440179
"""CS030512000026""",947,-0.588362
"""CS001515000291""",918,-0.599021
"""CS019414000068""",7698,1.893008
"""CS015515000222""",3081,0.196002


### P-060

レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [20]:
df_receipt.filter(
    pl.col.customer_id.str.starts_with('Z').not_()    
).group_by(
    'customer_id'
).agg(
    sum_amount=pl.col.amount.sum()
).with_columns(
    std_amount=(pl.col.sum_amount - pl.col.sum_amount.min()) / (pl.col.sum_amount.max() - pl.col.sum_amount.min())
).head(10)

customer_id,sum_amount,std_amount
str,i64,f64
"""CS040615000097""",808,0.032065
"""CS020513000237""",493,0.018379
"""CS037412000196""",522,0.019639
"""CS011415000291""",2427,0.102407
"""CS039412000034""",226,0.006778
"""CS008615000035""",679,0.02646
"""CS026512000023""",563,0.02142
"""CS004513000566""",1353,0.055744
"""CS010415000215""",7486,0.322211


## 演習問題 061 ~ 070

### P-061

レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [4]:
df_receipt.group_by('customer_id').agg(
    pl.col.amount.sum().alias('sum_amount'),
    (pl.col.amount.sum() + 0.5).log10().alias('log_amount')
).head()

customer_id,sum_amount,log_amount
str,i64,f64
"""CS014115000001""",1761,3.245883
"""CS013415000193""",6831,3.834516
"""CS028515000164""",5063,3.704451
"""CS023414000036""",6346,3.802534
"""CS033515000117""",4358,3.639337


### P-062

レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [5]:
df_receipt.group_by('customer_id').agg(
    pl.col.amount.sum().alias('sum_amount'),
    (pl.col.amount.sum() + 0.5).log().alias('log_amount')
).head()

customer_id,sum_amount,log_amount
str,i64,f64
"""CS032215000052""",1989,7.595639
"""CS010514000045""",1149,7.047082
"""CS003515000203""",436,6.078788
"""CS035512000032""",138,4.93087
"""CS021512000055""",2290,7.736525


### P-063

商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [7]:
df_product.select(
    pl.col.product_cd,
    pl.col.unit_price,
    pl.col.unit_cost,
    (pl.col.unit_price - pl.col.unit_cost).alias('unit_profit')
).head()

product_cd,unit_price,unit_cost,unit_profit
str,i64,i64,i64
"""P040101001""",198,149,49
"""P040101002""",218,164,54
"""P040101003""",230,173,57
"""P040101004""",248,186,62
"""P040101005""",268,201,67


### P-064 

商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [9]:
df_product.select(
    ((pl.col.unit_price - pl.col.unit_cost) / pl.col.unit_price).mean()
).item()

0.24911389885177007

### P-065

商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [2]:
df_product.select(
    pl.col.product_cd,
    pl.col.unit_price,
    pl.col.unit_cost,
    (pl.col.unit_cost / 0.7).floor().cast(pl.Int32).alias('new_price')
).with_columns(
    new_profit_rate=(pl.col.new_price - pl.col.unit_cost) / pl.col.new_price
).head(10)

product_cd,unit_price,unit_cost,new_price,new_profit_rate
str,i64,i64,i32,f64
"""P040101001""",198,149,212,0.29717
"""P040101002""",218,164,234,0.299145
"""P040101003""",230,173,247,0.299595
"""P040101004""",248,186,265,0.298113
"""P040101005""",268,201,287,0.299652
"""P040101006""",298,224,320,0.3
"""P040101007""",338,254,362,0.298343
"""P040101008""",420,315,450,0.3
"""P040101009""",498,374,534,0.299625


### P-066

商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [4]:
df_product.select(
    pl.col.product_cd,
    pl.col.unit_price,
    pl.col.unit_cost,
    (pl.col.unit_cost / 0.7).round().cast(pl.Int32).alias('new_price')
).with_columns(
    new_profit_rate=(pl.col.new_price - pl.col.unit_cost) / pl.col.new_price
).head(10)

product_cd,unit_price,unit_cost,new_price,new_profit_rate
str,i64,i64,i32,f64
"""P040101001""",198,149,213,0.300469
"""P040101002""",218,164,234,0.299145
"""P040101003""",230,173,247,0.299595
"""P040101004""",248,186,266,0.300752
"""P040101005""",268,201,287,0.299652
"""P040101006""",298,224,320,0.3
"""P040101007""",338,254,363,0.300275
"""P040101008""",420,315,450,0.3
"""P040101009""",498,374,534,0.299625


### P-067

商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [6]:
df_product.select(
    pl.col.product_cd,
    pl.col.unit_price,
    pl.col.unit_cost,
    (pl.col.unit_cost / 0.7).ceil().cast(pl.Int32).alias('new_price')
).with_columns(
    new_profit_rate=(pl.col.new_price - pl.col.unit_cost) / pl.col.new_price
).head(10)

product_cd,unit_price,unit_cost,new_price,new_profit_rate
str,i64,i64,i32,f64
"""P040101001""",198,149,213,0.300469
"""P040101002""",218,164,235,0.302128
"""P040101003""",230,173,248,0.302419
"""P040101004""",248,186,266,0.300752
"""P040101005""",268,201,288,0.302083
"""P040101006""",298,224,320,0.3
"""P040101007""",338,254,363,0.300275
"""P040101008""",420,315,451,0.301552
"""P040101009""",498,374,535,0.300935


### P-068

商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [10]:
df_product.select(
    pl.col.product_cd,
    pl.col.unit_price,
    (pl.col.unit_price * 1.1).floor().cast(pl.Int32).alias('tex_price')
).head(10)

product_cd,unit_price,tex_price
str,i64,i32
"""P040101001""",198,217
"""P040101002""",218,239
"""P040101003""",230,253
"""P040101004""",248,272
"""P040101005""",268,294
"""P040101006""",298,327
"""P040101007""",338,371
"""P040101008""",420,462
"""P040101009""",498,547


### P-069

レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [16]:
df_receipt.join(
    df_product, on='product_cd', how='left'
).group_by(
    'customer_id', maintain_order=True
).agg(
    sum_all=pl.col.amount.sum(),
    sum_07=pl.col.amount.filter(pl.col.category_major_cd == '07').sum()
).with_columns(
    sales_rate=pl.col.sum_07 / pl.col.sum_all
).head(10)

customer_id,sum_all,sum_07,sales_rate
str,i64,i64,f64
"""CS006214000001""",7364,4713,0.640005
"""CS008415000097""",1895,1337,0.705541
"""CS028414000014""",6222,3701,0.594825
"""ZZ000000000000""",12395003,6943009,0.560146
"""CS025415000050""",5736,3536,0.616457
"""CS003515000195""",5412,4581,0.846452
"""CS024514000042""",533,0,0.0
"""CS040415000178""",6149,4802,0.78094
"""CS027514000015""",2788,1475,0.529053


### P-070

レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [28]:
df_receipt.select('customer_id', 'sales_ymd').unique().join(
    df_customer, on='customer_id'
).select(
    pl.col.customer_id,
    pl.col.sales_ymd.cast(str).str.strptime(pl.Date, '%Y%m%d'),
    pl.col.application_date.cast(str).str.strptime(pl.Date, '%Y%m%d')
).with_columns(
    elapsed_days=(pl.col.sales_ymd - pl.col.application_date).dt.total_days()
).head()

customer_id,sales_ymd,application_date,elapsed_days
str,date,date,i64
"""CS012515000143""",2019-05-23,2015-05-13,1471
"""CS015415000120""",2017-10-01,2015-09-28,734
"""CS027414000084""",2019-08-17,2015-03-17,1614
"""CS014414000032""",2017-04-27,2015-03-14,775
"""CS019215000046""",2018-08-19,2015-07-18,1128


## 演習問題 071 ~ 080

### P-071

レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [37]:
df_receipt.select('customer_id', 'sales_ymd').unique(keep='first').join(
    df_customer, on='customer_id'
).select(
    pl.col.customer_id,
    pl.col.sales_ymd.cast(str).str.strptime(pl.Date, '%Y%m%d'),
    pl.col.application_date.cast(str).str.strptime(pl.Date, '%Y%m%d')
).with_columns(
    elapsed_months=(pl.col.sales_ymd.dt.year() - pl.col.application_date.dt.year()) * 12 +
                   (pl.col.sales_ymd.dt.month() - pl.col.application_date.dt.month())
).head()

customer_id,sales_ymd,application_date,elapsed_months
str,date,date,i32
"""CS040415000178""",2019-09-22,2015-06-27,51
"""CS025415000134""",2019-09-18,2015-07-20,50
"""CS026615000085""",2018-11-01,2015-08-02,39
"""CS038614000007""",2017-07-29,2015-05-12,26
"""CS015415000061""",2017-11-12,2015-05-28,30


### P-072

レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [38]:
df_receipt.select('customer_id', 'sales_ymd').unique(keep='first').join(
    df_customer, on='customer_id'
).select(
    pl.col.customer_id,
    pl.col.sales_ymd.cast(str).str.strptime(pl.Date, '%Y%m%d'),
    pl.col.application_date.cast(str).str.strptime(pl.Date, '%Y%m%d')
).with_columns(
    elapsed_years=pl.col.sales_ymd.dt.year() - pl.col.application_date.dt.year()
).head()

customer_id,sales_ymd,application_date,elapsed_years
str,date,date,i32
"""CS039414000052""",2018-05-06,2015-05-09,3
"""CS019412000185""",2018-02-24,2015-04-16,3
"""CS022414000065""",2019-03-12,2015-08-06,4
"""CS003515000131""",2019-02-15,2015-09-03,4
"""CS010513000187""",2019-08-08,2015-07-10,4


### P-073

レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [3]:
df_receipt.select('customer_id', 'sales_ymd').unique(keep='first').join(
    df_customer, on='customer_id'
).select(
    pl.col.customer_id,
    pl.col.sales_ymd.cast(str).str.strptime(pl.Date, '%Y%m%d'),
    pl.col.application_date.cast(str).str.strptime(pl.Date, '%Y%m%d')
).with_columns(
    elapsed_years=(pl.col.sales_ymd - pl.col.application_date).dt.total_seconds()
).head()

customer_id,sales_ymd,application_date,elapsed_years
str,date,date,i64
"""CS024514000042""",2018-12-05,2015-10-10,99532800
"""CS026515000042""",2019-06-03,2015-07-21,122083200
"""CS011115000004""",2019-07-18,2015-03-06,137808000
"""CS020814000004""",2018-04-18,2014-12-23,104716800
"""CS009415000022""",2018-05-01,2015-07-02,89337600


### P-074

レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [47]:
df_receipt.select('customer_id', 'sales_ymd').join(
    df_customer, on='customer_id'
).select(
    pl.col.sales_ymd.cast(str).str.strptime(pl.Date, '%Y%m%d'),
).with_columns(
    pl.col.sales_ymd.dt.truncate('1w').alias('monday')
).with_columns(
    (pl.col.sales_ymd - pl.col.monday).alias('elapsed_days').dt.total_days()
).head()

sales_ymd,monday,elapsed_days
date,date,i64
2018-11-03,2018-10-29,5
2018-11-18,2018-11-12,6
2017-07-12,2017-07-10,2
2018-08-21,2018-08-20,1
2019-06-05,2019-06-03,2


### P-075

顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [50]:
df_customer.sample(fraction=0.01).head(5)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,str,str,str,i64,str,str,str,i64,str
"""CS008702000012""","""飯島 輝信""","""0""","""男性""","""1947-05-04""",71,"""213-0033""","""神奈川県川崎市高津区下作延**********""","""S13008""",20150727,"""0-00000000-0"""
"""CS033403000041""","""大高 良介""","""0""","""男性""","""1977-07-16""",41,"""246-0038""","""神奈川県横浜市瀬谷区宮沢**********""","""S14033""",20150226,"""0-00000000-0"""
"""CS018314000102""","""亀山 さやか""","""1""","""女性""","""1987-12-09""",31,"""203-0041""","""東京都東久留米市野火止**********""","""S13018""",20171109,"""0-00000000-0"""
"""CS002313000272""","""本橋 理紗""","""1""","""女性""","""1983-05-30""",35,"""185-0024""","""東京都国分寺市泉町**********""","""S13002""",20160703,"""0-00000000-0"""
"""CS015602000035""","""小関 草太""","""0""","""男性""","""1954-05-28""",64,"""135-0043""","""東京都江東区塩浜**********""","""S13015""",20170807,"""0-00000000-0"""


### P-076

顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [9]:
df_customer.filter(
    (pl.int_range(pl.len()) < pl.len() * 0.1).shuffle().over('gender_cd') == 1
).group_by('gender_cd').agg(
    pl.len()
)

gender_cd,len
str,u32
"""1""",1792
"""0""",299
"""9""",108


### P-077

レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [12]:
df_receipt.group_by(
    'customer_id',
).agg(
    sum_amount=pl.col.amount.sum(),
    log_sum_amount=pl.col.amount.sum().log(),
).filter(
    (pl.col.log_sum_amount - pl.col.log_sum_amount.mean()) / pl.col.log_sum_amount.std() > 3
)

customer_id,sum_amount,log_sum_amount
str,i64,f64
"""ZZ000000000000""",12395003,16.332804


### P-078

レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [11]:
col = pl.col.sum_amount
q25 = col.quantile(0.25)
q75 = col.quantile(0.75)
iqr = q75 - q25
expr = (col > q75 + iqr * 1.5) | (col < q25 - iqr * 1.5)

df_receipt.filter(
    pl.col.customer_id.str.starts_with('Z').not_()
).group_by(
    'customer_id'
).agg(
    sum_amount=pl.col.amount.sum()
).filter(expr).head()

customer_id,sum_amount
str,i64
"""CS041411000001""",9694
"""CS035415000174""",14508
"""CS009415000047""",10648
"""CS024415000206""",14093
"""CS035415000213""",9306


### P-079

商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [12]:
df_product.select(
    pl.all().null_count()
)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
u32,u32,u32,u32,u32,u32
0,0,0,0,7,7


### P-080

商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [15]:
len(df_product), len(df_product.drop_nulls())

(10030, 10023)

## 演習問題 081 ~ 090

### P-081

単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [22]:
df_product.with_columns(
    pl.col.unit_price.fill_null(pl.col.unit_price.mean()),
    pl.col.unit_cost.fill_null(pl.col.unit_cost.mean())
).select(pl.all().null_count())

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


### P-082

単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [23]:
df_product.with_columns(
    pl.col.unit_price.fill_null(pl.col.unit_price.median()),
    pl.col.unit_cost.fill_null(pl.col.unit_cost.median())
).select(pl.all().null_count())

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


### P-083

単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [33]:
cols = pl.col('unit_price', 'unit_cost')
df_product.with_columns(
    pl.when(cols.is_null()).then(cols.median()).otherwise(cols).over('category_small_cd')
).select(pl.all().null_count())

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


### P-084

顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [20]:
df_customer.join(
    df_receipt.group_by('customer_id').agg(
        sales_amount_all=pl.col.amount.sum()
    ), 
    on='customer_id', how='left'
).join(
    df_receipt.filter((pl.col.sales_ymd // 10000) == 2019).group_by('customer_id').agg(
        sales_amount_2019=pl.col.amount.sum()
    ), 
    on='customer_id', how='left'
).select(
    'customer_id',
    pl.col.sales_amount_all.fill_null(0),
    pl.col.sales_amount_2019.fill_null(0)
).with_columns(
    sales_rate=(pl.col.sales_amount_2019 / pl.col.sales_amount_all).fill_nan(0)
).filter(
    pl.col.sales_amount_2019 > 0
).head()

customer_id,sales_amount_all,sales_amount_2019,sales_rate
str,i64,i64,f64
"""CS031415000172""",5088,2971,0.583923
"""CS015414000103""",3122,874,0.279949
"""CS011215000048""",3444,248,0.072009
"""CS029415000023""",5167,3767,0.72905
"""CS035415000029""",7504,5823,0.775986


### P-085

顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [25]:
df_loc = df_geocode.group_by(
    pl.col.postal_cd
).agg(
    pl.col.longitude.mean(),
    pl.col.latitude.mean()
)

df_customer_loc = df_customer.join(
    df_loc, on='postal_cd', how='left'
)
df_customer_loc.head()

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,longitude,latitude
str,str,str,str,str,i64,str,str,str,i64,str,f64,f64
"""CS021313000114""","""大野 あや子""","""1""","""女性""","""1981-04-29""",37,"""259-1113""","""神奈川県伊勢原市粟窪**********""","""S14021""",20150905,"""0-00000000-0""",139.31779,35.41358
"""CS037613000071""","""六角 雅彦""","""9""","""不明""","""1952-04-01""",66,"""136-0076""","""東京都江東区南砂**********""","""S13037""",20150414,"""0-00000000-0""",139.83502,35.67193
"""CS031415000172""","""宇多田 貴美子""","""1""","""女性""","""1976-10-04""",42,"""151-0053""","""東京都渋谷区代々木**********""","""S13031""",20150529,"""D-20100325-C""",139.68965,35.67374
"""CS028811000001""","""堀井 かおり""","""1""","""女性""","""1933-03-27""",86,"""245-0016""","""神奈川県横浜市泉区和泉町**********""","""S14028""",20160115,"""0-00000000-0""",139.4836,35.39125
"""CS001215000145""","""田崎 美紀""","""1""","""女性""","""1995-03-29""",24,"""144-0055""","""東京都大田区仲六郷**********""","""S13001""",20170605,"""6-20090929-2""",139.70775,35.54084


### P-086

085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

In [36]:
p1 = pl.col.latitude.radians()
p2 = pl.col.latitude_right.radians()
l1 = pl.col.longitude.radians()
l2 = pl.col.longitude.radians()
distance = (p1.sin() * p2.sin() + p1.cos() * p2.cos() * (l1 - l1).cos()).arccos() * 6371

df_customer_loc.join(
    df_store,
    left_on='application_store_cd',
    right_on='store_cd',
    how='left'
).with_columns(
    distance=distance
).select(
    'customer_id',
    pl.col.address.alias('customer_address'),
    pl.col.address_right.alias('store_address'),
    'distance'
).head()

customer_id,customer_address,store_address,distance
str,str,str,f64
"""CS021313000114""","""神奈川県伊勢原市粟窪**********""","""神奈川県伊勢原市伊勢原四丁目""",1.322108
"""CS037613000071""","""東京都江東区南砂**********""","""東京都江東区南砂一丁目""",0.783924
"""CS031415000172""","""東京都渋谷区代々木**********""","""東京都渋谷区初台二丁目""",0.269092
"""CS028811000001""","""神奈川県横浜市泉区和泉町**********""","""神奈川県横浜市瀬谷区二ツ橋町""",7.982684
"""CS001215000145""","""東京都大田区仲六郷**********""","""東京都大田区仲六郷二丁目""",1.168659


### P-087

顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [48]:
df_receipt_sum = df_receipt.group_by(
    'customer_id'
).agg(
    pl.col.amount.sum()
)

df_unique_customer = df_customer.join(
    df_receipt_sum, on='customer_id', how='left'
).with_columns(
    pl.col.amount.fill_null(0)
).group_by(
    'customer_name', 'postal_cd'
).agg(
    pl.col('customer_id', 'amount').sort_by('amount', 'customer_id', descending=[True, False]).first(),
)

df_unique_customer.head()

customer_name,postal_cd,customer_id,amount
str,str,str,i64
"""依田 満""","""276-0022""","""CS007403000016""",0
"""芦田 沙耶""","""223-0062""","""CS011215000048""",3444
"""堀口 陽子""","""242-0015""","""CS025412000147""",0
"""砂川 あさみ""","""259-1131""","""CS021313000025""",0
"""堤 明慶""","""226-0016""","""CS040413000257""",0


In [49]:
df_customer.shape[0], df_unique_customer.shape[0]

(21971, 21941)

### P-088

087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。

>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [53]:
df_customer_integration = df_customer.join(
    df_unique_customer.select(
        'customer_name', 'postal_cd', pl.col.customer_id.alias('integration_id')
    ),
    on=['customer_name', 'postal_cd'], 
    how='left',
)
df_customer_integration.head()

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,integration_id
str,str,str,str,str,i64,str,str,str,i64,str,str
"""CS021313000114""","""大野 あや子""","""1""","""女性""","""1981-04-29""",37,"""259-1113""","""神奈川県伊勢原市粟窪**********""","""S14021""",20150905,"""0-00000000-0""","""CS021313000114"""
"""CS037613000071""","""六角 雅彦""","""9""","""不明""","""1952-04-01""",66,"""136-0076""","""東京都江東区南砂**********""","""S13037""",20150414,"""0-00000000-0""","""CS037613000071"""
"""CS031415000172""","""宇多田 貴美子""","""1""","""女性""","""1976-10-04""",42,"""151-0053""","""東京都渋谷区代々木**********""","""S13031""",20150529,"""D-20100325-C""","""CS031415000172"""
"""CS028811000001""","""堀井 かおり""","""1""","""女性""","""1933-03-27""",86,"""245-0016""","""神奈川県横浜市泉区和泉町**********""","""S14028""",20160115,"""0-00000000-0""","""CS028811000001"""
"""CS001215000145""","""田崎 美紀""","""1""","""女性""","""1995-03-29""",24,"""144-0055""","""東京都大田区仲六郷**********""","""S13001""",20170605,"""6-20090929-2""","""CS001215000145"""


In [55]:
df_customer_integration.filter(
    pl.col.customer_id != pl.col.integration_id
).shape[0]

30

### P-089

売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [14]:
df_train, df_test = df_receipt.group_by(
    'customer_id'
).agg(
    pl.col.amount.sum()
).join(
    df_customer,
    on='customer_id',
    how='left'
).with_columns(
    split=pl.int_range(pl.len()).shuffle() < pl.len() * 0.2
).partition_by(
    'split'
)

df_train.shape[0], df_test.shape[0]

(6645, 1662)

### P-090

レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [43]:
df_tmp = df_receipt.group_by(
    (pl.col.sales_ymd // 100).alias('sales_ym')
).agg(
    sum_amount=pl.col.amount.sum()
).sort(
    by='sales_ym'
)

group = df_tmp.with_row_index().group_by_dynamic(
    pl.col.index.cast(pl.Int32), every='6i', period='18i', offset='0i'
)

dfs = [df for _, df in group][:3]
dfs = [df.with_columns(
    test_flag=(pl.int_range(pl.len()) > 12).cast(pl.UInt8)
).select(pl.exclude('index')) for df in dfs]

dfs

[shape: (18, 3)
 ┌──────────┬────────────┬───────────┐
 │ sales_ym ┆ sum_amount ┆ test_flag │
 │ ---      ┆ ---        ┆ ---       │
 │ i64      ┆ i64        ┆ u8        │
 ╞══════════╪════════════╪═══════════╡
 │ 201701   ┆ 902056     ┆ 0         │
 │ 201702   ┆ 764413     ┆ 0         │
 │ 201703   ┆ 962945     ┆ 0         │
 │ 201704   ┆ 847566     ┆ 0         │
 │ 201705   ┆ 884010     ┆ 0         │
 │ …        ┆ …          ┆ …         │
 │ 201802   ┆ 864128     ┆ 1         │
 │ 201803   ┆ 946588     ┆ 1         │
 │ 201804   ┆ 937099     ┆ 1         │
 │ 201805   ┆ 1004438    ┆ 1         │
 │ 201806   ┆ 1012329    ┆ 1         │
 └──────────┴────────────┴───────────┘,
 shape: (18, 3)
 ┌──────────┬────────────┬───────────┐
 │ sales_ym ┆ sum_amount ┆ test_flag │
 │ ---      ┆ ---        ┆ ---       │
 │ i64      ┆ i64        ┆ u8        │
 ╞══════════╪════════════╪═══════════╡
 │ 201707   ┆ 959205     ┆ 0         │
 │ 201708   ┆ 954836     ┆ 0         │
 │ 201709   ┆ 902037     ┆ 0   

## 演習問題 091~100

### P-091

顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [53]:
df_tmp = df_customer.join(
    df_receipt.group_by(
        pl.col.customer_id
    ).agg(
        sum_amount=pl.col.amount.sum()
    ),
    on='customer_id',
    how='left'
)

flag = pl.col.sum_amount.is_null().cast(pl.UInt8)

df_result = df_tmp.filter(
    pl.int_range(pl.len()).shuffle().over(flag) < pl.len().over(flag).min()
)

df_result.group_by(
    flag
).agg(
    count=pl.col.customer_id.count()
)

sum_amount,count
u8,u32
0,8306
1,8306


### P-092

顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

In [57]:
df_gender_std = df_customer.select('gender_cd', 'gender').unique()
df_gender_std

gender_cd,gender
str,str
"""9""","""不明"""
"""0""","""男性"""
"""1""","""女性"""


In [59]:
df_customer_std = df_customer.select(
    'customer_id',
    'customer_name',
    'gender_cd',
    'birth_day',
    'age',
    'postal_cd',
    'application_store_cd',
    'application_date',
    'status_cd'
)
df_customer_std.head()

customer_id,customer_name,gender_cd,birth_day,age,postal_cd,application_store_cd,application_date,status_cd
str,str,str,str,i64,str,str,i64,str
"""CS021313000114""","""大野 あや子""","""1""","""1981-04-29""",37,"""259-1113""","""S14021""",20150905,"""0-00000000-0"""
"""CS037613000071""","""六角 雅彦""","""9""","""1952-04-01""",66,"""136-0076""","""S13037""",20150414,"""0-00000000-0"""
"""CS031415000172""","""宇多田 貴美子""","""1""","""1976-10-04""",42,"""151-0053""","""S13031""",20150529,"""D-20100325-C"""
"""CS028811000001""","""堀井 かおり""","""1""","""1933-03-27""",86,"""245-0016""","""S14028""",20160115,"""0-00000000-0"""
"""CS001215000145""","""田崎 美紀""","""1""","""1995-03-29""",24,"""144-0055""","""S13001""",20170605,"""6-20090929-2"""


### P-093

商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [67]:
df_product_full = df_product.join(
    df_category,
    on='category_small_cd',
    how='inner'
).select(
    'product_cd',
    'category_major_cd',
    'category_major_name',
    'category_medium_cd',
    'category_medium_name',
    'category_small_cd',
    'category_small_name',
    'unit_price',
    'unit_cost'
)
df_product_full.head()

product_cd,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name,unit_price,unit_cost
str,str,str,str,str,str,str,i64,i64
"""P040101001""","""04""","""惣菜""","""0401""","""御飯類""","""040101""","""弁当類""",198,149
"""P040101002""","""04""","""惣菜""","""0401""","""御飯類""","""040101""","""弁当類""",218,164
"""P040101003""","""04""","""惣菜""","""0401""","""御飯類""","""040101""","""弁当類""",230,173
"""P040101004""","""04""","""惣菜""","""0401""","""御飯類""","""040101""","""弁当類""",248,186
"""P040101005""","""04""","""惣菜""","""0401""","""御飯類""","""040101""","""弁当類""",268,201


### P-094

093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [68]:
df_product_full.write_csv('./data/product_full.csv')

### P-095

093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [72]:
import io
buf = io.BytesIO()
df_product_full.write_csv(buf)
with open('./data/product_full_cp932.csv', 'wb') as f:
    f.write(buf.getvalue().decode('utf-8').encode('cp932'))

### P-096

093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [73]:
df_product_full.write_csv('./data/product_full_nohead.csv', include_header=False)

### P-097

094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

In [75]:
pl.read_csv('./data/product_full.csv').head()

product_cd,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name,unit_price,unit_cost
str,i64,str,i64,str,i64,str,i64,i64
"""P040101001""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",198,149
"""P040101002""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",218,164
"""P040101003""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",230,173
"""P040101004""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",248,186
"""P040101005""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",268,201


### P-098

096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

In [77]:
df_product_full.columns

['product_cd',
 'category_major_cd',
 'category_major_name',
 'category_medium_cd',
 'category_medium_name',
 'category_small_cd',
 'category_small_name',
 'unit_price',
 'unit_cost']

In [81]:
cols = ['product_cd',
 'category_major_cd',
 'category_major_name',
 'category_medium_cd',
 'category_medium_name',
 'category_small_cd',
 'category_small_name',
 'unit_price',
 'unit_cost']

pl.read_csv('./data/product_full_nohead.csv', has_header=False, new_columns=cols).head()

product_cd,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name,unit_price,unit_cost
str,i64,str,i64,str,i64,str,i64,i64
"""P040101001""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",198,149
"""P040101002""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",218,164
"""P040101003""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",230,173
"""P040101004""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",248,186
"""P040101005""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",268,201


### P-099

093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [82]:
df_product_full.write_csv('./data/product_full.tsv', separator='\t')

### P-100

099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。

> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

In [85]:
pl.read_csv('./data/product_full.tsv', separator='\t').head()

product_cd,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name,unit_price,unit_cost
str,i64,str,i64,str,i64,str,i64,i64
"""P040101001""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",198,149
"""P040101002""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",218,164
"""P040101003""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",230,173
"""P040101004""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",248,186
"""P040101005""",4,"""惣菜""",401,"""御飯類""",40101,"""弁当類""",268,201
